In [1]:
cd ..

In [2]:
from accidents_montreal import get_accident_df
from preprocess import init_spark
from road_network import get_road_df
from weather import add_weather_columns, extract_year_month_day
from pyspark.sql.functions import row_number, col, rank, avg, split, to_date, min

In [3]:
spark = init_spark()

# accidents

In [4]:
acc_df = get_accident_df(spark)

Skip fetching montreal accidents dataset: already downloaded
Skip extraction of accidents montreal dataframe: already done, reading from file


In [5]:
acc_df.columns

['NO_SEQ_COLL',
 'JR_SEMN_ACCDN',
 'DT_ACCDN',
 'CD_MUNCP',
 'NO_CIVIQ_ACCDN',
 'SFX_NO_CIVIQ_ACCDN',
 'BORNE_KM_ACCDN',
 'RUE_ACCDN',
 'TP_REPRR_ACCDN',
 'ACCDN_PRES_DE',
 'NB_METRE_DIST_ACCD',
 'CD_GENRE_ACCDN',
 'CD_SIT_PRTCE_ACCDN',
 'CD_ETAT_SURFC',
 'CD_ECLRM',
 'CD_ENVRN_ACCDN',
 'NO_ROUTE',
 'CD_CATEG_ROUTE',
 'CD_ETAT_CHASS',
 'CD_ASPCT_ROUTE',
 'CD_LOCLN_ACCDN',
 'CD_POSI_ACCDN',
 'CD_CONFG_ROUTE',
 'CD_ZON_TRAVX_ROUTR',
 'CD_PNT_CDRNL_ROUTE',
 'CD_PNT_CDRNL_REPRR',
 'CD_COND_METEO',
 'NB_VEH_IMPLIQUES_ACCDN',
 'NB_MORTS',
 'NB_BLESSES_GRAVES',
 'NB_BLESSES_LEGERS',
 'HEURE_ACCDN',
 'AN',
 'NB_VICTIMES_TOTAL',
 'GRAVITE',
 'REG_ADM',
 'MRC',
 'nb_automobile_camion_leger',
 'nb_camionLourd_tractRoutier',
 'nb_outil_equipement',
 'nb_tous_autobus_minibus',
 'nb_bicyclette',
 'nb_cyclomoteur',
 'nb_motocyclette',
 'nb_taxi',
 'nb_urgence',
 'nb_motoneige',
 'nb_VHR',
 'nb_autres_types',
 'nb_veh_non_precise',
 'NB_DECES_PIETON',
 'NB_BLESSES_PIETON',
 'NB_VICTIMES_PIETON',
 'NB_

In [6]:
dates_df = extract_year_month_day(acc_df.select('DT_ACCDN')
                                  .withColumn('date', to_date(col('DT_ACCDN'), format='yyyy/MM/dd'))).persist()
rows = dates_df.select('year').summary("min", "max").collect()
min_ = rows[0]['year']
max_ = rows[1]['year']
print(min_, max_)

2012 2017


In [7]:
# function get_random_date
import time
import datetime
import random
start = "01/01/" + str(min_)
end = "01/01/" + str(max_)
start_stamp = datetime.datetime.strptime(start, "%d/%m/%Y").timestamp()
end_stamp = datetime.datetime.strptime(end, "%d/%m/%Y").timestamp()
random.seed()
random_stamp = datetime.datetime.fromtimestamp(random.randint(start_stamp, end_stamp))
date = random_stamp.strftime("%d/%m/%Y")
hour = random.randint(0,12)
print(date, hour)

11/09/2014 7


In [8]:
# generate all dates from start to end
date = datetime.datetime.strptime(start, "%d/%m/%Y")

dates = list()
while(date != datetime.datetime.strptime(end, "%d/%m/%Y")): 
    date += datetime.timedelta(days=1)
    for i in range(24):
        dates.append((date.strftime("%d/%m/%Y"), i))

In [9]:
sc = spark.sparkContext

In [10]:
print("nb_dates: ",len(dates))
spark.createDataFrame(dates, ['date', 'hour']).limit(10).show()
dates_df = spark.createDataFrame(dates, ['date', 'hour']).persist()

nb_dates:  43848
+----------+----+
|      date|hour|
+----------+----+
|02/01/2012|   0|
|02/01/2012|   1|
|02/01/2012|   2|
|02/01/2012|   3|
|02/01/2012|   4|
|02/01/2012|   5|
|02/01/2012|   6|
|02/01/2012|   7|
|02/01/2012|   8|
|02/01/2012|   9|
+----------+----+



# location
location (route) => coordinates of a road 

loc_lat, loc_long

In [11]:
road_df = get_road_df(spark)

Skip fetching road network: already downloaded
Skip extraction of road network dataframe: already done, reading from file


In [12]:
from pyspark.sql.functions import udf, struct, rand
from pyspark.sql.types import *

to_tuple = udf(lambda x,y: (x,y))

print(road_df.select(['center_long', 'center_lat']).first())
road_df = road_df.select(['center_long', 'center_lat']).orderBy(rand()).persist()
print(road_df.first())

Row(center_long=-73.9142974, center_lat=45.4285574)
Row(center_long=-73.9023328, center_lat=45.6878661)


# weather
weather => year/month/day date + hour
just query the API

In [13]:
input_df = dates_df.crossJoin(road_df).persist()
test = input_df.sample(0.0000001).first().persist()
negative_samples = add_weather_columns(spark, test)

Py4JJavaError: An error occurred while calling o117.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 230, localhost, executor driver): java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:65)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:329)
	at java.nio.channels.Channels.writeFullyImpl(Channels.java:78)
	at java.nio.channels.Channels.writeFully(Channels.java:101)
	at java.nio.channels.Channels.access$000(Channels.java:61)
	at java.nio.channels.Channels$1.write(Channels.java:174)
	at org.apache.spark.storage.memory.RedirectableOutputStream.write(MemoryStore

In [ ]:
negative_samples.show()